In [1]:
from dbscan_parallel import *

24/05/15 09:27:11 WARN Utils: Your hostname, yaozys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.30.192.48 instead (on interface en0)
24/05/15 09:27:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/15 09:27:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from sklearn.datasets import make_blobs

# Generate random dataset with 3 dimensions
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 3  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)

In [3]:
X.shape

(100, 3)

In [5]:
np.max(X, axis=0)

array([ 4.5123324 , 10.41824487,  8.01217509])

In [6]:
np.min(X, axis=0)

array([-10.83328933,  -8.24949825,  -8.02990217])

In [34]:
def _get_dist(m, a, b) -> float:
    """
    for float comparison, set all distance value precision to 5
    :param: a: int; index of given point in data matrix
    :param: b: same as a
    """
    result = np.sqrt(np.power(m[b] - m[a], 2).sum())

    return round(result, 5)

def _get_distance_matrix(dataset):
    """
    Only once calculation will be on each point-pairs
    results will be stored in self.dist_m
    """
    num_p = dataset.shape[0]
    dist_m = np.zeros((num_p, num_p))
    for p_id in range(num_p):
        for q_id in range(p_id, num_p):
            dist = _get_dist(dataset, p_id, q_id)
            dist_m[q_id, p_id] = dist
            dist_m[p_id, q_id] = dist
    
    return dist_m

def _get_neighbors(i, dist_m, eps):
    # get neighbors of point i
    return np.where(dist_m[i] < eps)[0]

dist_m = _get_distance_matrix(X)


(100, 100)

In [64]:
n_partitions = (3,3,3)
tp_par = n_partitions
num_par = np.prod(tp_par)

# cut bins
bounds = np.concatenate(([np.min(X, axis=0)], [np.max(X, axis=0)]), axis=0)     # 2 * D
bounds_dim = bounds.T   # D * 2, 

In [68]:
bin_bounds = []
for i in range(len(tp_par)):
    dim_bins = np.linspace(*bounds_dim[i], tp_par[i]+1, endpoint=True)
    bin_bounds.append(dim_bins)
bin_bounds

[array([-10.83328933,  -5.71808209,  -0.60287484,   4.5123324 ]),
 array([-8.24949825, -2.02691721,  4.19566383, 10.41824487]),
 array([-8.02990217, -2.68254308,  2.664816  ,  8.01217509])]

In [78]:
lower_cdnts = [[low] for low in  bin_bounds[0][:-1]]
upper_cdnts = [[high] for high in bin_bounds[0][1:]]
lower_cdnts

[[-10.83328932819366], [-5.718082085392418], [-0.6028748425911772]]

In [79]:
for bound in bin_bounds[1:]:
    lower_tmp = []
    upper_tmp = []
    
    for bc in bound[:-1]:
        lower_tmp.extend([lc + [bc] for lc in lower_cdnts])
    lower_cdnts = lower_tmp
    
    for bc in bound[1:]:
        upper_tmp.extend([uc + [bc] for uc in upper_cdnts])
    upper_cdnts = upper_tmp
    
lower_bounds, upper_bounds = np.array(lower_cdnts), np.array(upper_cdnts)

In [80]:
# scatter points into bins with eps
indexed_data = []
# double loop to ensure border points could be given multiple partition ID
for id_pts in range(len(dataset)):     # index of point in dataset
    for id_ptt in range(num_par):
        if not (dataset[id_pts] > lower_bounds[id_ptt]).all():
            continue
        if not (dataset[id_pts] < upper_bounds[id_ptt]).all():
            continue
        indexed_data.append([id_ptt, id_pts])

In [81]:
indexed_data

[[25, 0],
 [15, 1],
 [15, 2],
 [25, 3],
 [2, 4],
 [2, 5],
 [25, 8],
 [2, 9],
 [2, 10],
 [2, 11],
 [25, 13],
 [2, 14],
 [2, 16],
 [25, 17],
 [2, 18],
 [15, 19],
 [24, 20],
 [16, 21],
 [25, 22],
 [2, 23],
 [15, 24],
 [25, 25],
 [24, 26],
 [16, 27],
 [2, 28],
 [15, 29],
 [2, 31],
 [2, 32],
 [25, 33],
 [25, 34],
 [25, 35],
 [15, 36],
 [2, 37],
 [15, 38],
 [24, 39],
 [25, 40],
 [25, 41],
 [15, 42],
 [15, 43],
 [2, 44],
 [24, 45],
 [2, 46],
 [25, 47],
 [15, 49],
 [2, 50],
 [25, 51],
 [15, 52],
 [24, 53],
 [2, 54],
 [25, 55],
 [26, 56],
 [2, 57],
 [25, 58],
 [15, 59],
 [2, 60],
 [25, 61],
 [2, 62],
 [25, 63],
 [25, 64],
 [24, 65],
 [2, 66],
 [2, 67],
 [15, 68],
 [2, 69],
 [2, 70],
 [15, 71],
 [25, 72],
 [2, 73],
 [24, 74],
 [25, 75],
 [15, 76],
 [24, 77],
 [25, 78],
 [2, 79],
 [15, 80],
 [25, 81],
 [15, 82],
 [25, 83],
 [15, 84],
 [15, 85],
 [2, 86],
 [25, 87],
 [2, 88],
 [25, 89],
 [2, 90],
 [25, 91],
 [15, 92],
 [15, 93],
 [15, 94],
 [2, 95],
 [25, 96],
 [25, 97],
 [15, 98],
 [15, 99]]

In [ ]:
neighbors = {}
for i in range(dist_m.shape[0]):
    i_neighbors = []
    for j in range(dist_m.shape[1]):
        if dist_m[i, j] < eps:
            i_neighbors.append(j)
    neighbors[i] = i_neighbors
    
neighbors

In [ ]:
len(rdd.keys().collect())

20

24/05/15 01:06:37 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 931161 ms exceeds timeout 120000 ms
24/05/15 01:06:37 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/15 01:06:38 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [25]:
dist_m

In [14]:
min_pts = 18
eps = 2
dataset = X
b_dataset = sc.broadcast(dataset)
b_eps = sc.broadcast(eps)
b_min_pts = sc.broadcast(min_pts)


partition_each_dim = (2, 2, 2)
grid_side_len = eps/np.sqrt(n_features)
gridnum_each_dim = [int((np.max(X[:, i]) - np.min(X[:, i])) / grid_side_len) for i in range(n_features)]

partitioned_rdd = spatial_partition(X, partition_each_dim, eps)
grid_rdd = spatial_partition(X, gridnum_each_dim, eps)


spatial_partition time cost: 51.702022552490234ms
spatial_partition time cost: 1682.992935180664ms


[[2048,
  [0,
   3,
   8,
   13,
   17,
   21,
   22,
   25,
   27,
   33,
   34,
   35,
   40,
   41,
   47,
   51,
   58,
   63,
   64,
   72,
   75,
   78,
   81,
   83,
   87,
   89,
   91,
   97]]]

In [7]:
@staticmethod
def local_dbscan(partioned_rdd, method='matrix', metric='euclidian'):

    dataset = np.array([b_dataset.value[idp] for idp in partioned_rdd])
    if method == 'matrix':
        dbscan_obj = MatrixDBSCAN(dataset, b_eps.value, b_min_pts.value, metric) 
    else:
        dbscan_obj = NaiveDBSCAN(dataset, b_eps.value, b_min_pts.value, metric) 
    dbscan_obj.predict()
    is_core_list = dbscan_obj._find_core_pts()
    
    return list(zip(partioned_rdd, is_core_list, dbscan_obj.tags))


local_tags = rdd.mapValues(lambda x: local_dbscan(x, method='matrix', metric='euclidian')).collect()

predict time cost: 0.9989738464355469ms                             (0 + 8) / 8]
predict time cost: 0.16808509826660156ms
predict time cost: 2.4399757385253906ms
predict time cost: 11.595964431762695ms
predict time cost: 14.004945755004883ms
predict time cost: 3.5741329193115234ms
predict time cost: 0.10275840759277344ms
predict time cost: 8.532047271728516ms
predict time cost: 0.09703636169433594ms
predict time cost: 65.52505493164062ms>                             (4 + 4) / 8]
predict time cost: 34.03902053833008ms
predict time cost: 0.8440017700195312ms
predict time cost: 108.0329418182373ms
predict time cost: 109.83109474182129ms
predict time cost: 0.06985664367675781ms
predict time cost: 0.19621849060058594ms
predict time cost: 92.9727554321289ms
predict time cost: 11.018037796020508ms
predict time cost: 5.685091018676758ms
predict time cost: 33.27608108520508ms


In [8]:
local_tags

[(24,
  [(0, 0, 2),
   (1, 1, 1),
   (2, 1, 1),
   (7, 0, 1),
   (15, 0, 2),
   (19, 1, 1),
   (20, 1, 1),
   (22, 1, 2),
   (24, 1, 1),
   (26, 1, 1),
   (27, 0, 2),
   (29, 1, 1),
   (33, 1, 2),
   (34, 0, 2),
   (35, 0, 2),
   (36, 1, 1),
   (38, 1, 1),
   (39, 1, 1),
   (40, 0, 2),
   (42, 1, 1),
   (43, 1, 1),
   (45, 0, 1),
   (47, 0, 2),
   (49, 1, 1),
   (52, 1, 1),
   (53, 0, 2),
   (55, 0, 2),
   (58, 0, 2),
   (59, 1, 1),
   (61, 0, 2),
   (63, 1, 2),
   (64, 1, 2),
   (65, 1, 1),
   (68, 1, 1),
   (71, 1, 1),
   (72, 0, 2),
   (74, 0, 1),
   (75, 0, 2),
   (76, 1, 1),
   (77, 1, 1),
   (80, 1, 1),
   (82, 1, 1),
   (83, 1, 2),
   (84, 1, 1),
   (85, 1, 1),
   (92, 1, 1),
   (93, 1, 1),
   (94, 0, 1),
   (96, 0, 2),
   (97, 0, 2),
   (98, 0, 1),
   (99, 1, 1)]),
 (16,
  [(2, 0, -2),
   (3, 1, 1),
   (8, 1, 1),
   (13, 1, 1),
   (15, 1, 1),
   (17, 1, 1),
   (19, 0, -2),
   (21, 1, 1),
   (22, 1, 1),
   (25, 1, 1),
   (27, 1, 1),
   (33, 1, 1),
   (34, 1, 1),
   (35, 1, 1),
 

In [9]:
@timeit
def merge(local_tags, dataset):
    global_tags = [UNKNOWN] * len(dataset)
    is_tagged = [0] * len(dataset)
    last_max_label = 0
    for local in local_tags:
        np_local = np.array(local[-1])
        np_local[:, -1] += last_max_label

        last_max_label = np.max(np_local[:, -1])
        
        # check and merge overlapped points
        tagged_indices = np.nonzero(is_tagged)[0]
        for tmp_i in range(len(np_local)):
            # should do tag check
            p_id, is_core, label = np_local[tmp_i]
            if p_id in tagged_indices and is_core==1:
                np_local[-1][np_local[-1]==label] = global_tags[p_id]
        
        # update global tags
        for p_id, is_core, label in np_local:
            if is_tagged[p_id]==1:
                continue
            global_tags[p_id] = label
            is_tagged[p_id] = 1
    return global_tags

result_tags = merge(local_tags, dataset)


merge time cost: 7.142066955566406ms


In [10]:
len(result_tags)

100

In [11]:
rdd.collect()

[[24,
  [0,
   1,
   2,
   7,
   15,
   19,
   20,
   22,
   24,
   26,
   27,
   29,
   33,
   34,
   35,
   36,
   38,
   39,
   40,
   42,
   43,
   45,
   47,
   49,
   52,
   53,
   55,
   58,
   59,
   61,
   63,
   64,
   65,
   68,
   71,
   72,
   74,
   75,
   76,
   77,
   80,
   82,
   83,
   84,
   85,
   92,
   93,
   94,
   96,
   97,
   98,
   99]],
 [16,
  [2,
   3,
   8,
   13,
   15,
   17,
   19,
   21,
   22,
   25,
   27,
   33,
   34,
   35,
   36,
   38,
   40,
   45,
   47,
   49,
   52,
   53,
   55,
   56,
   63,
   64,
   68,
   71,
   72,
   75,
   76,
   78,
   80,
   81,
   82,
   83,
   84,
   85,
   87,
   89,
   91,
   93,
   94,
   96,
   97]],
 [25,
  [0,
   2,
   3,
   8,
   12,
   13,
   15,
   17,
   19,
   21,
   22,
   25,
   27,
   33,
   34,
   35,
   36,
   38,
   40,
   41,
   45,
   47,
   49,
   51,
   52,
   53,
   55,
   56,
   58,
   61,
   63,
   64,
   68,
   71,
   72,
   75,
   76,
   78,
   80,
   81,
   82,
   83,
   84,
   85,
  

In [16]:
from sklearn.cluster import DBSCAN

# Create an instance of the DBSCAN algorithm
dbscan = DBSCAN(eps=eps, min_samples=min_pts, metric='euclidean')

# Fit the DBSCAN algorithm to the data
labels = dbscan.fit_predict(X)


In [18]:
from sklearn.metrics import adjusted_mutual_info_score, adjusted_rand_score


# Calculate the Mutual Information (MI) score
mi_score = adjusted_mutual_info_score(result_tags, labels)

# Calculate the Adjusted Rand Index (ARI)
ari_score = adjusted_rand_score(result_tags, labels)

print("Mutual Information (MI) score:", mi_score)
print("Adjusted Rand Index (ARI):", ari_score)


Mutual Information (MI) score: 0.8014438842685137
Adjusted Rand Index (ARI): 0.7146010924553108


In [19]:
labels

array([0, 1, 1, 0, 2, 2, 2, 1, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 2, 1, 1, 0,
       0, 2, 1, 0, 1, 0, 2, 1, 2, 2, 2, 0, 0, 0, 1, 2, 1, 1, 0, 0, 1, 1,
       2, 1, 2, 0, 2, 1, 2, 0, 1, 0, 2, 0, 0, 2, 0, 1, 2, 0, 2, 0, 0, 1,
       2, 2, 1, 2, 2, 1, 0, 2, 1, 0, 1, 1, 0, 2, 1, 0, 1, 0, 1, 1, 2, 0,
       2, 0, 2, 0, 1, 1, 1, 2, 0, 0, 1, 1])

In [21]:
np.array(result_tags)

array([2, 1, 1, 3, 6, 8, 6, 1, 3, 6, 6, 6, 4, 3, 6, 2, 6, 3, 8, 1, 1, 3,
       2, 6, 1, 3, 1, 2, 8, 1, 8, 6, 6, 2, 2, 2, 1, 6, 1, 1, 2, 4, 1, 1,
       8, 1, 6, 2, 6, 1, 6, 4, 1, 2, 8, 2, 3, 6, 2, 1, 6, 2, 6, 2, 2, 1,
       8, 8, 1, 6, 8, 1, 2, 8, 1, 2, 1, 1, 3, 6, 1, 3, 1, 2, 1, 1, 6, 3,
       6, 3, 6, 3, 1, 1, 1, 6, 2, 2, 1, 1])